In [2]:
import numpy as numpy

#Kezdeti j1, j2, J paramétereket itt kell megadni.
j1=3/2
j2=5/2
J=2

if (J < abs(j1-j2) or J > j1+j2):
    raise Exception("A paraméterek értéke nem megfelelő! Kezdeti feltétel: |j1-j2|<=J<=j1+j2.")
    
# Függvény a 8. egyenletben szereplő q-val jelölt gyökös kifejezéshez.    
def sq(m1, m2):
    return -numpy.sqrt((j2*(j2+1)-m2*(m2+1))/(j1*(j1+1)-m1*(m1-1)))

# A CG együtthatók értékeit egy 3D-s arrayben fogom tárolni m1, m2, M paramétereknek megfelelően.
# Az array indexelése a következő: CG[m1][m2][M].
# Természetesen a paraméterek értékeit majd át kell váltani indexekre (pl: m1 -> m1_index).
# Például ha M értéke -2..2 lehet, akkor az M=-2-es értékhez tartozó CG együttható az [m1][m2][0] indexű helyre kerül.
# Így a feladat végén könnyen le tudjuk kérdezni, hogy az adott CG érték milyen m1, m2, M számhármashoz tartozik.
# Első lépésként létrehozom a megfelelő méretű arrayt.

m1_length = int(2*j1+1)
m2_length = int(2*j2+1)
M_length = int(2*J+1)
CG = numpy.zeros((m1_length,m2_length,M_length))

# A következő feladat, hogy kiszámoljuk CG(j1,j1,j2,J-j1|JJ) értékét. 
# Ehhez kelleni fog az összes megengedett m1,m2 kombináció a 9. egyenletnek megfelelően.
# m1=j1 értéktől kezdem, ezt csökkentem, ebből megkapható m2: J-j1, és mindig ellenőrzöm, hogy m2 nem-e nagyobb mint j2.
# A range(m1_length) miatt m1 értéke -j1 értékéig csökkenhet, ezért azt külön nem kell ellenőirzni, hogy
# esetleg kisebb lesz-e.
szumm = 1
ci = 1
for i in range(m1_length):
    m1 = j1-i
    m2 = J-m1
    if m2 < j2:
        ci = ci * sq(m1,m2)**2
        szumm = ci + szumm
c = numpy.sqrt(1/szumm)
m1_index = int(j1 + j1)      #m1 = j1 megfelelő indexe
m2_index = int(J - j1 + j2)  #m2 = J-j1 megfelelő indexe
M_index = int(J + J)
CG[m1_index][m2_index][M_index] = c

# Ezután kiszámoljuk az összes CG(j1,m1,j2,m2|JJ) értéket a 8. egyenlet szerint.
# Szintén a kiindulási érték m1=j1 és m2=J-j1.
for i in range(m1_length):
    m1 = j1-i
    m2 = J-m1
    m1_index = int(m1 + j1)
    m2_index = int(m2 + j2)
    M_index = int(J + J)
    if (m2+1) <= j2:
        CG[m1_index-1][m2_index+1][M_index] = sq(m1,m2) * CG[m1_index][m2_index][M_index]

# Most következik a lefelé léptetés: CG(j1,m1,j2,m2|J M-1).
# A 10. egyenletet használjuk.
# Itt M értékét fogom léptetni mindig 1-gyel lefelé, és megnézem minden egyes lépéshez
# az összes lehetséges m1,m2 kombinációt.
for iM in range(M_length-1):
    M=J-iM
    M_index = int(M + J)
    if M_length > 1:
        for i in range(m1_length):
            m1 = j1-i
            m2 = M-m1-1
            m1_index = int(m1 + j1)
            m2_index = int(m2 + j2)
            if m2 <= j2:
                if abs(m1+1) > j1:
                    C1 = 0
                else:
                    C1 = numpy.sqrt(j1*(j1+1)-m1*(m1+1))/numpy.sqrt(J*(J+1)-M*(M-1))*CG[m1_index+1][m2_index][M_index]
                if abs(m2+1) > j2:
                    C2 = 0
                else:
                    C2 = numpy.sqrt(j2*(j2+1)-m2*(m2+1))/numpy.sqrt(J*(J+1)-M*(M-1))*CG[m1_index][m2_index+1][M_index]
                CG[m1_index][m2_index][M_index-1] = C1 + C2
                
#Kinyomtatom az eredményeket.
print('j1=' + str(j1) + ', j2=' + str(j2) + ', J=' + str(J))
for M_index in range(M_length):
    M = M_index - J
    for m1_index in range(m1_length):
        m1 = m1_index - j1
        for m2_index in range(m2_length):
            m2 = m2_index - j2
            C = CG[m1_index][m2_index][M_index]
            if abs(C) > 10**(-6):
                print('M=' + str(M) + ', m1=' + str(m1) + ', m2=' + str(m2) + ', C=' + str(C))

j1=1.5, j2=2.5, J=2
M=-2, m1=-1.5, m2=-0.5, C=0.37796447300922725
M=-2, m1=-0.5, m2=-1.5, C=-0.6172133998483679
M=-2, m1=0.5, m2=-2.5, C=0.6900655593423541
M=-1, m1=-1.5, m2=0.5, C=0.5669467095138409
M=-1, m1=-0.5, m2=-0.5, C=-0.545544725589981
M=-1, m1=0.5, m2=-1.5, C=0.15430334996209183
M=-1, m1=1.5, m2=-2.5, C=0.5976143046671969
M=0, m1=-1.5, m2=1.5, C=0.6546536707079771
M=0, m1=-0.5, m2=0.5, C=-0.2672612419124244
M=0, m1=0.5, m2=-0.5, C=-0.2672612419124244
M=0, m1=1.5, m2=-1.5, C=0.6546536707079771
M=1, m1=-1.5, m2=2.5, C=0.5976143046671967
M=1, m1=-0.5, m2=1.5, C=0.15430334996209194
M=1, m1=0.5, m2=0.5, C=-0.5455447255899809
M=1, m1=1.5, m2=-0.5, C=0.5669467095138407
M=2, m1=-0.5, m2=2.5, C=0.6900655593423541
M=2, m1=0.5, m2=1.5, C=-0.6172133998483675
M=2, m1=1.5, m2=0.5, C=0.3779644730092272
